In [27]:
import torch
import tensorflow as tf
import pprint
from datasets import load_dataset
import openai
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from IPython.display import display, HTML
import time

In [28]:
print(torch.cuda.is_available())  # Should return True if CUDA is available

True


In [29]:
# Reading API Key from file
API_KEY = open("open_ai_API.txt", "r").read().strip()
openai.api_key = API_KEY

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
data = load_dataset("generaleoley/manim-codegen", split='train')
# print(data)

In [52]:
def similarity_f(embedded_open_gen,text, model):
    """
    Computes the cosine similarity between the embeddings of two text inputs.

    Args:
    -----
    text1 (str): 
        The first input text that will be encoded and compared.
    
    text2 (str): 
        The second input text that will be encoded and compared.

    Returns:
    --------
    float: 
        A floating-point value representing the cosine similarity between the two text embeddings. The value ranges
        between -1 and 1:
        - 1 indicates that the texts are identical in terms of the embedding space.
        - 0 indicates that the texts are orthogonal (no similarity).
        - -1 indicates maximum dissimilarity in terms of the embedding space.
        
    """
    
    
    embedding = model.encode(text, convert_to_tensor=True)
    return util.pytorch_cos_sim(embedded_open_gen, embedding).item()

def create_message_for_comparison(code_snippet, paragraphs):
    """
    Compares the prompts with the highest cosine similarity to ultimately find the correct prompt for the code 

    Args:
    -----
    code_snippet (str):
        A string containing the Python code that will be compared against the provided paragraphs.
    
    paragraphs (list of str):
        A list of paragraphs (prompts) to be compared with the Python code. Each paragraph is a potential
        candidate that could have generated the provided code snippet.

    Returns:
    --------
    list:
        A list containing two dictionaries, formatted as messages:
        1. A system message that defines the role and behavior of the model.
        2. A user message that contains the code snippet and the paragraphs for comparison.
    """
    # System message to guide GPT's behavior
    system_message = {
        "role": "system",
        # "content": "You are an assistant that compares Python code snippets with paragraphs and finds the most similar match."
        "content": "You are an assistant that compares prompts to find the one which most likely generated the provided code"
    }
    
    # User message that provides the code and paragraphs
    user_message_content = f"""
    Here is the Python code snippet:
    
    ```python
    {code_snippet}
    ```

    Below are the prompts {len(paragraphs)}. Compare each prompt with the provided code snippet and determine which prompt is the most similar to the code. Provide the index of the most similar prompt and print that prompt under ** ** without paragraph index or anything else.
    """
    
    # Append each paragraph with its index
    for i, paragraph in enumerate(paragraphs, start=1):
        user_message_content += f"\nParagraph {i}: {paragraph}"
    
    user_message = {
        "role": "user",
        "content": user_message_content
    }
    
    return [system_message, user_message]

# Isolates output and runs a search for it through the dataset to find its corresponding index in the dataset
def searching(response, reference):
    search_phrase = response[20:-15]  # Skip the first 15 characters
    # Search for the phrase in the 'query' column and store the row number
    filtered_rows_with_indices_query = [(index, example) for index, example in enumerate(data) if search_phrase.lower() in example['query'].lower()]

    print("Reference =", reference,",  No. of instances =",len(filtered_rows_with_indices_query))

    # Display the filtered rows with their corresponding row numbers
    for index, row in filtered_rows_with_indices_query:
        print(f"Row {index}:")
        # print(row['query'])

    result = [(reference, index, row['query']) for index, row in filtered_rows_with_indices_query]
    

def searching2(response, reference):
    """
    Isolates the output and runs a search through the dataset to find its corresponding index in the dataset.

    Args:
    -----
    response (str): 
        The response text to search for in the dataset. The phrase is extracted by removing the first and last 15 characters.

    reference (int): 
        The reference number associated with this search.

    Returns:
    --------
    list: 
        A list of tuples where each tuple contains the reference and the index of the matching row.
    """
    
    search_phrase = response[20:-15]  # Skip the first 15 characters and the last 15 characters
    
    # Search for the phrase in the 'query' column and store the row index
    filtered_rows_with_indices_query = [index for index, example in enumerate(data) if search_phrase.lower() in example['query'].lower()]

    # Only return reference and index, not the full example
    result = [(reference, index) for index in filtered_rows_with_indices_query]
    
    return result

    

def printer(printing_content):
    """
    Displays content in a formatted HTML output with word wrapping.

    This function takes a string input and displays it using IPython's display capabilities, ensuring that
    the text is wrapped appropriately (i.e., long lines of text won't overflow the display area). It uses 
    HTML's `pre-wrap` style to preserve spaces and newlines while also wrapping text for better readability.

    Args:
    -----
    printing_content (str):
        The content to be displayed. This can be any text or string that should be printed with word wrapping.
    
    Returns:
    --------
    None:
        The function outputs the wrapped content to the notebook using IPython's `display` and `HTML` functions,
        but it does not return any value.
    """
    # Display the output with word wrapping
    wrapped_output = f"<div style='white-space: pre-wrap;'>{printing_content}</div>"
    return display(HTML(wrapped_output))

# Isolates output and runs a search for it through the dataset to find its corresponding index in the dataset


In [66]:
start_time = time.time()
#Choosing a specific code from the dataset by giving value of 'reference', change value for each code 

sorted_set = []

for i in range(100):
    
    reference = i
    code_string = data[reference]['answer']
    formatted_code_string = code_string.replace('\\n', '\n')
    # print(formatted_code_string)

    # Prepare the messages for the Chat API which will be used later for cosine similarity
    messages = [
        # {"role": "system", "content": "You are a helpful assistant that performs semantic analysis."},
        {"role": "user", "content": f"Explain the purpose of this code wihtout getting into technical details, \n\nParagraph 1:\n{formatted_code_string}"},

        {"role": "user", "content": f"In detail, explain what is happening with the visuals,  \n\nParagraph 1:\n{formatted_code_string} "}
    ]

    # Make the API call
    response = openai.chat.completions.create(
        model="gpt-4o-mini",  # Or another model as appropriate
        messages=messages,
        max_tokens=700,  # Adjust based on your needs
        temperature=0.01  # Adjust for more creativity or precision
    )
    open_gen = response.choices[0].message.content
    # printer(open_gen)
    
    top_similarities = []  # List to store (similarity, index) tuples
    top_n = 10  # Number of top similarities to keep
    model1 = SentenceTransformer('all-MiniLM-L6-v2').to(device)

    for i in range(len(data)):
        query_response = data[i]['query']
        open_gen_encoded = model1.encode(open_gen, convert_to_tensor=True)
        score = similarity_f(open_gen_encoded, query_response, model=model1)

        # Add the current score and index as a dictionary to the list
        top_similarities.append({'score': score, 'index': i, 'query': data[i]['query']})  # Storing the score and index in a dictionary

        # Sort the list by similarity score in descending order and keep only the top N
        top_similarities = sorted(top_similarities, key=lambda x: x['score'], reverse=True)[:top_n]

    
    all_scores = [item['score'] for item in top_similarities]
    all_indices = [item['index'] for item in top_similarities]
    all_queries = [item['query'] for item in top_similarities]
        
    messages2 = create_message_for_comparison(code_string, all_queries)

    # Make the API call to GPT-4o-mini for comparison
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages2,
        max_tokens=700,
        temperature=0.01
    )

    # Extract and return the most similar query from the response
    response_content = response.choices[0].message.content
    # printer(response_content)
    
    searched_index = searching2(response_content, reference)
    print(searched_index)

    sorted_set.append(searched_index)


end_time = time.time()
print(f"Exectution time :",end_time - start_time)

[(0, 771)]
[(1, 990)]
[]
[(3, 615)]
[(4, 737)]
[(5, 878)]
[(6, 771)]
[(7, 385)]
[(8, 751)]
[(9, 321)]
[]
[(11, 385)]
[(12, 818)]
[(13, 156)]
[(14, 627)]
[(15, 452)]
[(16, 57)]
[(17, 1018)]
[(18, 92)]
[(19, 362)]
[(20, 755)]
[(21, 540)]
[(22, 558)]
[(23, 49)]
[(24, 797)]
[(25, 196)]
[(26, 667)]
[(27, 535)]
[(28, 937)]
[(29, 918)]
[(30, 117)]
[(31, 308)]
[(32, 314)]
[(33, 287)]
[(34, 447)]
[(35, 116)]
[(36, 223)]
[(37, 556)]
[(38, 84)]
[(39, 193)]
[(40, 206)]
[(41, 804)]
[(42, 798)]
[(43, 663)]
[(44, 328)]
[(45, 643)]
[(46, 631)]
[(47, 933)]
[(48, 1001)]
[(49, 771)]
[(50, 246)]
[(51, 40)]
[]
[(53, 777)]
[(54, 833)]
[(55, 225)]
[(56, 532)]
[(57, 927)]
[]
[(59, 576)]
[(60, 351)]
[(61, 303)]
[]
[]
[(64, 613)]
[(65, 229)]
[(66, 146)]
[(67, 64)]
[(68, 537)]
[(69, 641)]
[(70, 810)]
[(71, 813)]
[(72, 699)]
[(73, 615)]
[(74, 817)]
[(75, 62)]
[(76, 782)]
[(77, 919)]
[(78, 751)]
[(79, 707)]
[(80, 118)]
[(81, 254)]
[(82, 445)]
[(83, 862)]
[(84, 978)]
[(85, 843)]
[(86, 552)]
[(87, 540)]
[(88, 732)]


In [67]:
sorted_set

[[(0, 771)],
 [(1, 990)],
 [],
 [(3, 615)],
 [(4, 737)],
 [(5, 878)],
 [(6, 771)],
 [(7, 385)],
 [(8, 751)],
 [(9, 321)],
 [],
 [(11, 385)],
 [(12, 818)],
 [(13, 156)],
 [(14, 627)],
 [(15, 452)],
 [(16, 57)],
 [(17, 1018)],
 [(18, 92)],
 [(19, 362)],
 [(20, 755)],
 [(21, 540)],
 [(22, 558)],
 [(23, 49)],
 [(24, 797)],
 [(25, 196)],
 [(26, 667)],
 [(27, 535)],
 [(28, 937)],
 [(29, 918)],
 [(30, 117)],
 [(31, 308)],
 [(32, 314)],
 [(33, 287)],
 [(34, 447)],
 [(35, 116)],
 [(36, 223)],
 [(37, 556)],
 [(38, 84)],
 [(39, 193)],
 [(40, 206)],
 [(41, 804)],
 [(42, 798)],
 [(43, 663)],
 [(44, 328)],
 [(45, 643)],
 [(46, 631)],
 [(47, 933)],
 [(48, 1001)],
 [(49, 771)],
 [(50, 246)],
 [(51, 40)],
 [],
 [(53, 777)],
 [(54, 833)],
 [(55, 225)],
 [(56, 532)],
 [(57, 927)],
 [],
 [(59, 576)],
 [(60, 351)],
 [(61, 303)],
 [],
 [],
 [(64, 613)],
 [(65, 229)],
 [(66, 146)],
 [(67, 64)],
 [(68, 537)],
 [(69, 641)],
 [(70, 810)],
 [(71, 813)],
 [(72, 699)],
 [(73, 615)],
 [(74, 817)],
 [(75, 62)],
 [(7

In [68]:
i = 5
printer(all_queries[i])
printer(all_indices[i])